In [38]:
import pyTigerGraph as tg 
import pandas as pd
conn = tg.TigerGraphConnection(graphname="SCOTUS_Graph")

In [39]:
res = conn.runInstalledQuery("justiceCaseLinks", params={"justiceID": 105})

In [40]:
res[0]["@@justiceEmbedding"].keys()

dict_keys(['AScalia'])

In [41]:
caseEmbeddings = res[0]["@@caseEmbeddings"]

In [42]:
caseVotes = pd.DataFrame.from_dict(res[0]["@@caseVote"], orient="index").reset_index().rename(columns={"index":"caseId", 0:"vote"})

In [43]:
caseVotes["vote"].value_counts()

VOTED_IN_FAVOR_OF_PLAINTIFF    1781
VOTED_IN_FAVOR_OF_DEFENDANT    1080
Name: vote, dtype: int64

In [44]:
caseEmbeddings = pd.DataFrame.from_dict(caseEmbeddings, orient="index").reset_index().rename(columns={"index":"caseId"})

In [45]:
data = caseVotes.merge(caseEmbeddings, on="caseId")

In [46]:
X = data[[i for i in range(0, 200)]]
data["vote"] = data["vote"].astype("category")
data["vote_code"] = data["vote"].cat.codes
y = data["vote_code"]

In [47]:
data["vote"].head()

0    VOTED_IN_FAVOR_OF_PLAINTIFF
1    VOTED_IN_FAVOR_OF_PLAINTIFF
2    VOTED_IN_FAVOR_OF_PLAINTIFF
3    VOTED_IN_FAVOR_OF_PLAINTIFF
4    VOTED_IN_FAVOR_OF_PLAINTIFF
Name: vote, dtype: category
Categories (2, object): ['VOTED_IN_FAVOR_OF_DEFENDANT', 'VOTED_IN_FAVOR_OF_PLAINTIFF']

In [48]:
data["vote"].cat.codes.head()

0    1
1    1
2    1
3    1
4    1
dtype: int8

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = LogisticRegression(class_weight="balanced")
clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test), target_names=["Defendant", "Plaintiff"]))



              precision    recall  f1-score   support

   Defendant       0.44      0.62      0.52       217
   Plaintiff       0.69      0.53      0.60       356

    accuracy                           0.56       573
   macro avg       0.57      0.57      0.56       573
weighted avg       0.60      0.56      0.57       573

